**Dénes Csala**  
University of Bristol, 2022  

Based on *Elements of Data Science* ([Allen B. Downey](https://allendowney.com), 2021) and *Python Data Science Handbook* ([Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/), 2018)

License: [MIT](https://mit-license.org/)

# Loading financial data into _pandas_

Install _Yahoo Finance_. If you put a `!` in the first character of a cell, it becomes a _Linux_ command. The cell below would install the `yfinance` _python package_, using the `pip` package manager tool. However, when you run the cell for the second time (in _Colab_ the packages you install persist for about 8 hours) it is already installed - so you get only `Requirement already satisfied` messages.

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 14.8 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf

In [ ]:
nvda = yf.Ticker("NVDA").history(period='5y')
tsla = yf.Ticker("TSLA").history(period='5y')
ryaay = yf.Ticker("RYAAY").history(period='5y')
wizz = yf.Ticker("WIZZ.L").history(period='5y')

Now we have downloaded the data for four stocks, computer graphics company nVidia `NVDA` and electric vehicle manufacturer Tesla `TSLA`, and low-cost airlines RyanAir `RYAAY` and WizzAir `WIZZ.L` for the past 5 years.

The responses returned are _pandas_ `DataFrames`. They contain [OHLC](https://www.investopedia.com/terms/o/ohlcchart.asp) data, but we only need the `Close` columns this time. Let us also give them names.

In [ ]:
nvda=nvda[['Close']]
nvda['Name']='NVDA'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
nvda.head()

,Close,Name
Date,,
2016-11-28,23.224274,NVDA
2016-11-29,23.012047,NVDA
2016-11-30,22.752928,NVDA
2016-12-01,21.627621,NVDA
2016-12-02,21.827509,NVDA


This is what we want. However surely, there is a more efficient way to automating this, if we have multiple stocks.

In [ ]:
dfs=[] #create empty list of dataframes
for x in ['NVDA','TSLA','RYAAY','WIZZ.L']:
  df = yf.Ticker(x).history(period='5y')
  df=df[['Close']]
  df['Name']=x
  dfs.append(df) #append newly download and formatted dataframe to our list of dtaaframes

Great. Now we have a list of `DataFrame`s, each containing the closing stock price and the stock name, for the past 5 years. 

# DataFrame combination

## Concatenation

In [ ]:
len(dfs)

4

In [ ]:
dfs[0].head(2)

,Close,Name
Date,,
2016-11-28,23.224270,NVDA
2016-11-29,23.012047,NVDA


In [ ]:
dfs[1].head(2)

,Close,Name
Date,,
2016-11-28,39.223999,TSLA
2016-11-29,37.914001,TSLA


We can combine `DataFrames` by stacking them on top of each other using `concat`. They must have the same `column` names (otherwise, empty columns will be created and filled with `NaN`s). The `pd.concat` function accepts only `list []` arguments - therefore, the `DataFrame`s to be combined have to be in the format `[dfA, dfB]`.

In [ ]:
import pandas as pd
pd.concat([ dfs[1],dfs[2] ])

,Close,Name
Date,,
2016-11-28,39.223999,TSLA
2016-11-29,37.914001,TSLA
2016-11-30,37.880001,TSLA
2016-12-01,36.375999,TSLA
2016-12-02,36.293999,TSLA
...,...,...
2021-11-19,105.889999,RYAAY
2021-11-22,102.750000,RYAAY
2021-11-23,104.000000,RYAAY


In [ ]:
pd.concat(dfs)

,Close,Name
Date,,
2016-11-28,23.224270,NVDA
2016-11-29,23.012047,NVDA
2016-11-30,22.752930,NVDA
2016-12-01,21.627615,NVDA
2016-12-02,21.827509,NVDA
...,...,...
2021-11-23,4300.000000,WIZZ.L
2021-11-24,4352.000000,WIZZ.L
2021-11-25,4399.000000,WIZZ.L


## Joining

Sometimes, the dataframes to be joined need to end up next to each other, a "_horizontal_ `concat`". This is called `join`. The `DataFrames` to be combined must have the same index. They must _not_ have any matching `column` names - though these can be renamed automatically using `lsuffix` or `rsuffix`.

In [ ]:
ryan=dfs[2]
wizz=dfs[3]
nvda=dfs[0]
ryan.join(wizz,rsuffix='_right').join(nvda,rsuffix='_nn')

,Close,Name,Close_right,Name_right,Close_nn,Name_nn
Date,,,,,,
2016-11-28,82.300003,RYAAY,NaN,NaN,23.224270,NVDA
2016-11-29,82.339996,RYAAY,1693.0,WIZZ.L,23.012047,NVDA
2016-11-30,79.839996,RYAAY,1676.0,WIZZ.L,22.752930,NVDA
2016-12-01,80.260002,RYAAY,1737.0,WIZZ.L,21.627615,NVDA
2016-12-02,80.669998,RYAAY,1753.0,WIZZ.L,21.827509,NVDA
...,...,...,...,...,...,...
2021-11-19,105.889999,RYAAY,4173.0,WIZZ.L,329.850006,NVDA
2021-11-22,102.750000,RYAAY,4184.0,WIZZ.L,319.559998,NVDA
2021-11-23,104.000000,RYAAY,4300.0,WIZZ.L,317.459991,NVDA


Let's do something smarter:

In [ ]:
dfz=pd.DataFrame() #initialise empty DataFrame
for x in dfs:
  stock_name=x['Name'].values[0]
  stock_name=stock_name.replace('.','')
  x=x[['Close']]
  x.columns=[stock_name]
  dfz=x.join(dfz)

In [ ]:
dfz.head()

,WIZZL,RYAAY,TSLA,NVDA
Date,,,,
2016-11-29,1693.0,82.339996,37.914001,23.012047
2016-11-30,1676.0,79.839996,37.880001,22.752930
2016-12-01,1737.0,80.260002,36.375999,21.627615
2016-12-02,1753.0,80.669998,36.293999,21.827509
2016-12-05,1762.0,82.129997,37.360001,22.673958


Ready to export. In _CSV_:

In [ ]:
dfz.to_csv('my_stocks.csv')

To _JSON_.

In [ ]:
import json

In [ ]:
json_list_of_dicts=list(df.T.to_dict().values())
open('my_stocks.json','w').write(json.dumps(json_list_of_dicts))

46827